In [1]:
import findspark

In [2]:
findspark.init("/usr/local/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F

In [9]:
spark = SparkSession.builder \
.appName("Spark YARN Example") \
.master("local[2]") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/29 04:56:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [11]:
order_items_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('order_items.csv')
orders_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('orders.csv')
products_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('products.csv')
categories_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('categories.csv')
departments_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('departments.csv')
customers_df = spark.read.format('csv').option('header',True).option("inferSchema", True).load('customers.csv')

In [12]:
order_items_df.show(5)

+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|            1|               1|               957|                1|           299.98|               299.98|
|            2|               2|              1073|                1|           199.99|               199.99|
|            3|               2|               502|                5|            250.0|                 50.0|
|            4|               2|               403|                1|           129.99|               129.99|
|            5|               4|               897|                2|            49.98|                24.99|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
only showi

In [13]:
orders_df.show(5)

+-------+--------------------+---------------+---------------+
|orderId|           orderDate|orderCustomerId|    orderStatus|
+-------+--------------------+---------------+---------------+
|      1|2013-07-25 00:00:...|          11599|         CLOSED|
|      2|2013-07-25 00:00:...|            256|PENDING_PAYMENT|
|      3|2013-07-25 00:00:...|          12111|       COMPLETE|
|      4|2013-07-25 00:00:...|           8827|         CLOSED|
|      5|2013-07-25 00:00:...|          11318|       COMPLETE|
+-------+--------------------+---------------+---------------+
only showing top 5 rows



In [14]:
products_df.show(5)

+---------+-----------------+--------------------+------------------+------------+--------------------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|
+---------+-----------------+--------------------+------------------+------------+--------------------+
|        1|                2|Quest Q64 10 FT. ...|              null|       59.98|http://images.acm...|
|        2|                2|Under Armour Men'...|              null|      129.99|http://images.acm...|
|        3|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        4|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        5|                2|Riddell Youth Rev...|              null|      199.99|http://images.acm...|
+---------+-----------------+--------------------+------------------+------------+--------------------+
only showing top 5 rows



In [15]:
categories_df.show(5)

+----------+--------------------+-------------------+
|categoryId|categoryDepartmentId|       categoryName|
+----------+--------------------+-------------------+
|         1|                   2|           Football|
|         2|                   2|             Soccer|
|         3|                   2|Baseball & Softball|
|         4|                   2|         Basketball|
|         5|                   2|           Lacrosse|
+----------+--------------------+-------------------+
only showing top 5 rows



In [16]:
departments_df.show(5)

+------------+--------------+
|departmentId|departmentName|
+------------+--------------+
|           2|       Fitness|
|           3|      Footwear|
|           4|       Apparel|
|           5|          Golf|
|           6|      Outdoors|
+------------+--------------+
only showing top 5 rows



In [17]:
customers_df.show(5)

+----------+-------------+-------------+-------------+----------------+--------------------+------------+-------------+---------------+
|customerId|customerFName|customerLName|customerEmail|customerPassword|      customerStreet|customerCity|customerState|customerZipcode|
+----------+-------------+-------------+-------------+----------------+--------------------+------------+-------------+---------------+
|         1|      Richard|    Hernandez|    XXXXXXXXX|       XXXXXXXXX|  6303 Heather Plaza| Brownsville|           TX|          78521|
|         2|         Mary|      Barrett|    XXXXXXXXX|       XXXXXXXXX|9526 Noble Embers...|   Littleton|           CO|          80126|
|         3|          Ann|        Smith|    XXXXXXXXX|       XXXXXXXXX|3422 Blue Pioneer...|      Caguas|           PR|            725|
|         4|         Mary|        Jones|    XXXXXXXXX|       XXXXXXXXX|  8324 Little Common|  San Marcos|           CA|          92069|
|         5|       Robert|       Hudson|    XXXX

### HW 1

In [18]:
order_items_df.select("orderItemOrderId").distinct().count()

57431

### HW 2

In [19]:
order_items_df.count()

172198

In [20]:
orders_df.count()

68883

### HW 3

In [21]:
dist = orders_df.select("orderStatus").distinct()

In [22]:
dist.show()

+---------------+
|    orderStatus|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+



In [23]:
canceled_orders_df = orders_df.filter(F.col('orderStatus')=='CANCELED')

In [24]:
canceled_orders_df.count()

1428

In [25]:
main_df = canceled_orders_df.join(order_items_df, canceled_orders_df['orderId']==order_items_df['orderItemOrderId'], how='inner')

In [26]:
main_df = main_df.join(products_df, main_df['orderItemProductId']==products_df['productId'], how='inner')

In [27]:
main_df.count()

3519

In [28]:
res_df = main_df.groupby('productName').agg(F.sum('orderItemSubTotal').alias('total_sales_amount')).orderBy('total_sales_amount', ascending=False)

In [29]:
res_df.show(10)

+--------------------+------------------+
|         productName|total_sales_amount|
+--------------------+------------------+
|Field & Stream Sp...|134393.27999999968|
|Perfect Fitness P...|  85785.7000000001|
|Nike Men's Free 5...| 80691.92999999996|
|Diamondback Women...| 80094.66000000027|
|Pelican Sunstream...| 66196.68999999986|
|Nike Men's Dri-FI...|           65750.0|
|Nike Men's CJ Eli...| 60705.32999999982|
|O'Brien Men's Neo...|58126.740000000005|
|Under Armour Girl...| 26153.45999999997|
|LIJA Women's Eyel...|            2145.0|
+--------------------+------------------+
only showing top 10 rows



In [30]:
res_df.write.mode("overwrite").parquet('/user/root/canceled_products.parquet')

In [31]:
! hdfs dfs -ls /user/root

Found 10 items
drwxr-xr-x   - root supergroup          0 2024-10-29 04:51 /user/root/.sparkStaging
drwxr-xr-x   - root supergroup          0 2024-10-29 00:48 /user/root/canceled_categories.parquet
drwxr-xr-x   - root supergroup          0 2024-10-29 04:57 /user/root/canceled_products.parquet
-rw-r--r--   1 root supergroup       1074 2024-10-28 22:50 /user/root/categories.csv
-rw-r--r--   1 root supergroup     953847 2024-10-28 22:50 /user/root/customers.csv
drwxr-xr-x   - root supergroup          0 2024-10-28 18:02 /user/root/datasets
-rw-r--r--   1 root supergroup         88 2024-10-28 22:50 /user/root/departments.csv
-rw-r--r--   1 root supergroup    5408988 2024-10-28 22:50 /user/root/order_items.csv
-rw-r--r--   1 root supergroup    2999990 2024-10-28 22:50 /user/root/orders.csv
-rw-r--r--   1 root supergroup     174196 2024-10-28 22:50 /user/root/products.csv


### HW 4

In [32]:
main2_df = main_df.join(categories_df, main_df['productCategoryId']==categories_df['categoryId'], how='inner')

In [33]:
main2_df.count()

3519

In [34]:
res2_df = main2_df.groupby('categoryName').agg(F.sum('orderItemSubTotal').alias('total_sales_amount')).orderBy('total_sales_amount', ascending=False)

In [35]:
res2_df.show(10)

+--------------------+------------------+
|        categoryName|total_sales_amount|
+--------------------+------------------+
|             Fishing|134393.27999999968|
|              Cleats|  85785.7000000001|
|    Cardio Equipment| 81351.92999999996|
|    Camping & Hiking| 80094.66000000027|
|        Water Sports| 66196.68999999986|
|     Women's Apparel|           65750.0|
|      Men's Footwear| 60705.32999999982|
|Indoor/Outdoor Games|58126.740000000005|
|       Shop By Sport|27423.439999999973|
|         Electronics|5685.4999999999945|
+--------------------+------------------+
only showing top 10 rows



In [36]:
res2_df.write.mode("overwrite").parquet('/user/root/canceled_categories.parquet')

In [37]:
! hdfs dfs -ls /user/root

Found 10 items
drwxr-xr-x   - root supergroup          0 2024-10-29 04:51 /user/root/.sparkStaging
drwxr-xr-x   - root supergroup          0 2024-10-29 04:57 /user/root/canceled_categories.parquet
drwxr-xr-x   - root supergroup          0 2024-10-29 04:57 /user/root/canceled_products.parquet
-rw-r--r--   1 root supergroup       1074 2024-10-28 22:50 /user/root/categories.csv
-rw-r--r--   1 root supergroup     953847 2024-10-28 22:50 /user/root/customers.csv
drwxr-xr-x   - root supergroup          0 2024-10-28 18:02 /user/root/datasets
-rw-r--r--   1 root supergroup         88 2024-10-28 22:50 /user/root/departments.csv
-rw-r--r--   1 root supergroup    5408988 2024-10-28 22:50 /user/root/order_items.csv
-rw-r--r--   1 root supergroup    2999990 2024-10-28 22:50 /user/root/orders.csv
-rw-r--r--   1 root supergroup     174196 2024-10-28 22:50 /user/root/products.csv


### HW 5

In [38]:
total_sales_df = orders_df.join(order_items_df, orders_df['orderId']==order_items_df['orderItemOrderId'], how='inner')

In [39]:
total_sales_df.count()

172198

In [40]:
highest_sale = total_sales_df.groupBy(
    F.year("orderDate").alias("year"),
    F.month("orderDate").alias("month")
).agg(
    F.sum("orderItemSubTotal").alias("total_sales")
).orderBy(F.desc("total_sales")).first()

In [41]:
highest_sale

Row(year=2013, month=11, total_sales=3168656.030000238)

In [42]:
turkish_months = [
    "Ocak", "Şubat", "Mart", "Nisan", "Mayıs", "Haziran", "Temmuz", "Ağustos", "Eylül", "Ekim", "Kasım", "Aralık"
]

In [43]:
year = highest_sale.year
month = turkish_months[highest_sale.month - 1] 
total_sale = highest_sale.total_sales

In [45]:
year, month, total_sale

(2013, 'Kasım', 3168656.030000238)

### HW 6

In [46]:
sales_by_week_day = total_sales_df.groupBy(
    F.date_format("orderDate", "u").cast("int").alias("week_day")
).agg(
    F.sum("orderItemSubTotal").alias("total_sales")
).orderBy(F.desc("total_sales")).first()

In [47]:
sales_by_week_day

Row(week_day=5, total_sales=5172864.230001052)

In [48]:
turkish_days = [
    "Pazartesi","Salı","Çarşamba","Perşembe","Cuma","Cumartesi","Pazar"
]

In [49]:
week_day = turkish_days[sales_by_week_day.week_day - 1]
total_sale = sales_by_week_day.total_sales

In [50]:
week_day, total_sale

('Cuma', 5172864.230001052)

### HW 7

In [51]:
all_df = order_items_df.join(orders_df, order_items_df['orderItemOrderId']==orders_df['orderId'], how='inner')

In [52]:
all_df = all_df.join(customers_df, all_df['orderCustomerId']==customers_df['customerId'], how='inner')

In [53]:
all_df = all_df.join(products_df, all_df['orderItemProductId']==products_df['productId'], how='inner')

In [54]:
all_df = all_df.join(categories_df, all_df['productCategoryId']==categories_df['categoryId'], how='inner')

In [55]:
all_df = all_df.join(departments_df, all_df['categoryDepartmentId']==departments_df['departmentId'], how='inner')

In [56]:
all_df.count()

172198

In [57]:
all_df.printSchema()

root
 |-- orderItemName: integer (nullable = true)
 |-- orderItemOrderId: integer (nullable = true)
 |-- orderItemProductId: integer (nullable = true)
 |-- orderItemQuantity: integer (nullable = true)
 |-- orderItemSubTotal: double (nullable = true)
 |-- orderItemProductPrice: double (nullable = true)
 |-- orderId: integer (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- orderCustomerId: integer (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- customerId: integer (nullable = true)
 |-- customerFName: string (nullable = true)
 |-- customerLName: string (nullable = true)
 |-- customerEmail: string (nullable = true)
 |-- customerPassword: string (nullable = true)
 |-- customerStreet: string (nullable = true)
 |-- customerCity: string (nullable = true)
 |-- customerState: string (nullable = true)
 |-- customerZipcode: integer (nullable = true)
 |-- productId: integer (nullable = true)
 |-- productCategoryId: integer (nullable = true)
 |-- productName: string

In [62]:
jdbcUrl = f"jdbc:postgresql://postgresql:5432/test1?user=train&password=Ankara06"

In [65]:
all_df.write.jdbc(url=jdbcUrl, 
                            table='retail_all', 
                            mode='overwrite', 
                            properties={'driver':'org.postgresql.Driver'})